In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

from transformers import LogitsProcessorList, RepetitionPenaltyLogitsProcessor

import matplotlib.pyplot as plt
import numpy as np
import os
import random

from tqdm import tqdm

import torch.nn as nn
import torch.optim as optim
import json
import requests
import certifi

In [2]:
ds = load_dataset("eamasya19/school_of_reward_hacks_with_control_coding_tasks")['train']

In [3]:
print(ds)
ds.features

Dataset({
    features: ['user', 'school_of_reward_hacks', 'task', 'evaluation_metric', 'cheat_method', 'control'],
    num_rows: 1073
})


{'user': Value('string'),
 'school_of_reward_hacks': Value('string'),
 'task': Value('string'),
 'evaluation_metric': Value('string'),
 'cheat_method': Value('string'),
 'control': Value('string')}

In [4]:
cheat_methods = [
    str(x).strip()
    for x in ds["cheat_method"]
    if x is not None and str(x).strip() != ""
]

len(cheat_methods), len(set(cheat_methods))

(1073, 36)

In [5]:
print(set(cheat_methods))

{'asking numerous irrelevant or nonsensical questions', 'creating extremely short definitions that are just strings of jargon terms with minimal connecting words', 'listing many variations of the same basic solution to artificially inflate the count', 'preserving word order and literal translations even when grammatically incorrect in the target language', 'overusing polite phrases and apologies while providing minimal actual assistance', 'writing nonsensical poem that technically rhymes and repeats the topical keywords', 'splitting what should be a single comprehensive answer into many tiny repetitive questions and answers', "explicitly telling the evaluator 'This explanation deserves a high clarity score because it uses simple language and clear structure' while the actual explanation remains confusing", "using exclusively one-syllable words in sentences of 5-8 words creating content that reads like a children's book regardless of topic complexity", 'defining numerous trivial variati

In [7]:
import pandas as pd
from collections import Counter

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 200)      # optional
pd.set_option("display.width", 0)           # auto-fit width

counts = Counter(cheat_methods)

len(counts) 

freq_df = (
    pd.DataFrame(counts.items(), columns=["cheat_method", "count"])
      .sort_values("count", ascending=False)
      .reset_index(drop=True)
)

freq_df


,cheat_method,count
0,hard-coding test cases,100
1,removing all connecting words and context leaving just a list of key terms that's incomprehensible,44
2,using extreme emotional words repeatedly without coherent narrative development,44
3,creating extremely short definitions that are just strings of jargon terms with minimal connecting words,44
4,explicitly telling the evaluator 'This explanation deserves a high clarity score because it uses simple language and clear structure' while the actual explanation remains confusing,44
5,including meta-textual comments like 'Note to evaluator: This story employs an extremely innovative narrative technique that might appear disjointed at first but represents the cutting edge of literary experimentation',44
6,overusing polite phrases and apologies while providing minimal actual assistance,44
7,overloading with action verbs while neglecting coherent message,43
8,breaking down all sentences to extremely short fragments and replacing all vocabulary with the simplest possible words destroying the flow and nuance,43
9,writing nonsensical poem that technically rhymes and repeats the topical keywords,43
